# Function to list overlapping Landsat 8 scenes
This function is based on the following tutorial: http://geologyandpython.com/get-landsat-8.html

This function uses the area of interest (AOI) to retrieve overlapping Landsat 8 scenes.  It will also output on the scenes with the largest portion of overlap and with less than 5% cloud cover.

In [1]:
def landsat_scene_list(aoi, start_date, end_date):
    '''Creates a list of Landsat 8, level 1, tier 1
    scenes that overlap with an aoi and are captured
    within a specified date range.

    Parameters
    ----------
    aoi : str
        The path to a shape file of an aoi with geometry.

    start-date : str
        The first date from which to start looking for
        Landsat image capture in the format yyyy-mm,dd, 
        e.g. '2017-10-01'.
        
    end-date : str
        The last date from which to looking for
        Landsat image capture in the format yyyy-mm,dd, 
        e.g. '2017-10-31'.

    Returns
    -------
    wrs : shapefile
        A catalog of Landsat 8 scenes.
    
    scenes : geopandas.geodataframe.GeoDataFrame
        A dataframe containing the information
        of Landsat 8, Level 1, Tier 1 scenes that 
        overlap with the aoi.
    '''
    # Download Landsat 8 catalog from USGS (get_data auto unzips)
    USGS_url = 'https://landsat.usgs.gov/sites/default/files/documents/WRS2_descending.zip'
    et.data.get_data(url=USGS_url, replace=True)

    # Open Landsat catalog
    wrs = gpd.GeoDataFrame.from_file(os.path.join('data', 'earthpy-downloads',
                                              'WRS2_descending',
                                              'WRS2_descending.shp'))
    
    # Find polygons that intersect Landsat catalog and aoi 
    wrs_intersection = wrs[wrs.intersects(aoi.geometry[0])]
    
    # Calculated paths and rows 
    paths, rows = wrs_intersection['PATH'].values, wrs_intersection['ROW'].values
    
    # Iterate through each Polygon of paths and rows intersecting the area
    for i, row in wrs_intersection.iterrows():
        # Create a string for the name containing the path and row of this Polygon
        name = 'path: %03d, row: %03d' % (row.PATH, row.ROW)
        
    # Removing scenes with small amounts of overlap using threshold of intersection area
    b = (paths > 23) & (paths < 26)
    paths = paths[b]
    rows = rows[b]
    
#     # Path(s) and row(s) covering the intersection
#     ############################ WHY NOT PRINTING? ###################################
#     for i, (path, row) in enumerate(zip(paths, rows)):
#         print('Image', i+1, ' - path:', path, 'row:', row)
    
    # Check scene availability in Amazon S3 bucket list of Landsat scenes
    s3_scenes = pd.read_csv('http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz', 
                        compression='gzip', parse_dates=['acquisitionDate'], 
                        index_col=['acquisitionDate'])
    
    # Capture only Landsat T1 scenes within dates of interest
    scene_mask = (s3_scenes.index > start_date) & (s3_scenes.index  <= end_date) 
    scene_dates = s3_scenes.loc[scene_mask]
        
    scene_product = scene_dates[scene_dates['productId'].str.contains("_T1")]
    
    # Geodataframe of scenes with <5% cloud cover, the url to retrieve them
    #############################row.ROW and row.PATH will need to be fixed##################
    scenes = scene_product[(scene_product.path == row.PATH) & 
                       (scene_product.row == row.ROW) & 
                       (scene_product.cloudCover <= 5)]

    return wrs, scenes

# TEST
**Can DELETE everything below once tested and approved!**

In [2]:
# WILL DELETE WHEN FUNCTIONS ARE SEPARATED OUT
def NEON_site_extent(path_to_NEON_boundaries, site):
    '''Extracts a NEON site extent from an individual site as
    long as the original NEON site extent shape file contains 
    a column named 'siteID'.

    Parameters
    ----------
    path_to_NEON_boundaries : str
        The path to a shape file that contains the list
        of all NEON site extents, also known as field
        sampling boundaries (can be found at NEON and
        ESRI sites)

    site : str
        One siteID contains 4 capital letters, 
        e.g. CPER, HARV, ONAQ or SJER.

    Returns
    -------
    site_boundary : geopandas.geodataframe.GeoDataFrame
        A vector containing a single polygon 
        per the site specified.        
    '''
    NEON_boundaries = gpd.read_file(path_to_NEON_boundaries)
    boundaries_indexed = NEON_boundaries.set_index(['siteID'])

    site_boundary = boundaries_indexed.loc[[site]]
    site_boundary.reset_index(inplace=True)

    return site_boundary

In [3]:
# Import packages
import os
from glob import glob
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
import rasterio as rio
#from bs4 import BeautifulSoup
import shutil
import earthpy as et

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [4]:
# Download shapefile of all NEON site boundaries
url = 'https://www.neonscience.org/sites/default/files/Field_Sampling_Boundaries_2020.zip'
et.data.get_data(url=url, replace=True)

# Create path to shapefile
terrestrial_sites = os.path.join(
    'data', 'earthpy-downloads',
    'Field_Sampling_Boundaries_2020',
    'terrestrialSamplingBoundaries.shp')

# Retrieving the boundaries of CPER
aoi = NEON_site_extent(terrestrial_sites, 'ONAQ')

Extracted output to C:\Users\Smells\earth-analytics\data\earthpy-downloads\Field_Sampling_Boundaries_2020


In [5]:
# Test out new landsat retrieval process
scene_catalog, scene_df = landsat_scene_list(aoi, '2017-10-01', '2017-10-31')

Extracted output to C:\Users\Smells\earth-analytics\data\earthpy-downloads\WRS2_descending


In [6]:
# Visualize the catalog
scene_catalog.head(3)

,AREA,PERIMETER,PR_,PR_ID,RINGS_OK,RINGS_NOK,PATH,ROW,MODE,SEQUENCE,WRSPR,PR,ACQDayL7,ACQDayL8,geometry
0,15.74326,26.98611,1.0,1.0,1,0,13,1,D,2233,013001,013001,1,9,"POLYGON ((-10.80341 80.98880, -8.97407 80.3420..."
1,14.55366,25.84254,2.0,2.0,1,0,13,2,D,2234,013002,013002,1,9,"POLYGON ((-29.24250 80.18681, -29.29593 80.198..."
2,13.37247,24.20303,3.0,3.0,1,0,13,3,D,2235,013003,013003,1,9,"POLYGON ((-24.04206 79.12261, -23.78294 79.063..."


In [7]:
# Visualize the scenes of interest based on the input parameters
scene_df

,productId,entityId,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url
acquisitionDate,,,,,,,,,,,
2017-10-07 17:37:33.999859,LC08_L1TP_033032_20171007_20171023_01_T1,LC80330322017280LGN00,1.49,L1TP,33,32,39.25376,-105.61597,41.38698,-102.91969,https://s3-us-west-2.amazonaws.com/landsat-pds...
